In [1]:
# Helper function. Adds vectors and b, with a coefficient (1-d) for a and d for b
def add_vectors(vector_a, vector_b, ca, cb):
    keys = set(vector_a.keys()) | set(vector_b.keys())
    result = dict()
    for c in keys:
        p_a = vector_a.get(c)
        if p_a == None:
            p_a = 0.0
        p_b = vector_b.get(c)
        if p_b == None:
            p_b = 0.0
        result[c] = ca * p_a + cb * p_b
    return result

# test
def test_add_vectors():
    ta = {"a":0.6, "b":0.4}
    tb = {"b":1.0}
    print add_vectors(ta, tb, 0.8, 0.2)

test_add_vectors()

{'a': 0.48, 'b': 0.52}


In [2]:
from math import fabs

# Helper function. Computes the absolute difference between vectors and b
# We use it to test convergence of pagerank computation
def diff_vectors(vector_a, vector_b):
    keys = set(vector_a.keys()) | set(vector_b.keys())
    result = dict()
    for c in keys:
        p_a = vector_a.get(c)
        if p_a == None:
            p_a = 0.0
        p_b = vector_b.get(c)
        if p_b == None:
            p_b = 0.0
        result[c] = fabs(p_a - p_b)
    return result

# test
def test_diff_vectors():
    ta = {"a":0.6, "b":0.4}
    tb = {"b":1.0}
    print diff_vectors(ta, tb)

test_diff_vectors()

{'a': 0.6, 'b': 0.6}


In [3]:
# Helper function. Takes as input a vector/dictionary, where we have nodes and counts
# and returns a probability vector
def normalize_vector(vector):
    norm = sum(vector.values())
    return { v: 1.0*vector[v]/norm for v in vector}

# test
def test_normalize_vector():
    ta = {"a":6, "b":4}
    print normalize_vector(ta)
    
test_normalize_vector()

{'a': 0.6, 'b': 0.4}


In [4]:
# Helper function. Takes as input a vector/dictionary, where we have nodes and probabilities
# (Implicit assumption that the probabilities add up to 1.0)
# Then removes the specified node, and renormalizes the remaining probabilities
# Warning: Will not work if the removed vector contains a probability of 1.0
def remove_normalize(vector, node_to_remove):
    prob = vector.get(node_to_remove)
    return { v: 1.0*vector[v]/(1.0-prob) for v in vector if v != node_to_remove}

# test
def test_normalize():
    ta = {"a":0.5, "b":0.4, "c":0.1}
    print remove_normalize(ta, "a")
    
test_normalize()

{'c': 0.2, 'b': 0.8}


In [5]:
# Helper function. Computes the logodds between the probabilities in two vectors
from math import log

# Computes the log-odds of two probabilities. 
# The s is a smoothing factor: With 0 we have no smoothing 
# (note: s=0 will cause an error if there are events with 0 probability in either vector)
# The n is the total number of nodes
def logodds(pa, pb, s, n):
    pa = 0.0 if pa == None else pa
    pb = 0.0 if pb == None else pb
    a = (pa + s) / ( 1.0 + s*n)
    b = (pb + s) / ( 1.0 + s*n)
    return  log(a,2) - log(b,2)

def logodds_vector(vector_a, vector_b, s):
    keys = set(vector_a.keys()) | set(vector_b.keys())
    n = len(keys)
    return { k: logodds(vector_a.get(k), vector_b.get(k), s, n) for k in keys}

# test
def test_logodds():
    ta = {"a":0.25, "b":0.75}
    tb = {"a":0.75, "b":0.25}
    print logodds_vector(ta, tb, 0)

    ta = {"a":0.5, "b":0.5}
    tb = {"a":0.25, "b":0.25, "c":0.25, "d": 0.25}
    print logodds_vector(ta, tb, 1)
    

test_logodds()


{'a': -1.584962500721156, 'b': 1.584962500721156}
{'a': 0.26303440583379367, 'c': -0.3219280948873622, 'b': 0.26303440583379367, 'd': -0.3219280948873622}


In [6]:
# The Good Vibes model defines the probability of a vibe being noticed as 
# i->j = count(i,j)/sum(count(i,*)) * count(i,j)/sum(count(*,j))
# We represent the adjacency matrix as a dictionary of dictionaries
# i: {j: count(i,j)/sum(count(i,*)) * count(i,j)/sum(count(*,j)) }
def get_good_vibes_adjacency_matrix(msg_counts):
    
    # We first compute the from/to normalizing values for each node
    to_counts = dict()
    from_counts = dict()
    for n,m,c in msg_counts:
        
        # We will ignore self-sending messages
        #if (n==m):
        #    continue
        
        count_n = to_counts.get(n)
        if count_n == None:
            count_n = 0.0
        to_counts[n] = c + count_n
    
        count_m = from_counts.get(m)
        if count_m == None:
            count_m = 0.0
        from_counts[m] = c + count_m

    # We compute the weights using the Good Vibes formula
    adjacency_matrix = dict()
    for n,m,c in msg_counts:
        edges_n = adjacency_matrix.get(n)
        if edges_n == None:
            edges_n = dict()
        edges_n[m] = (1.0*c/to_counts[n]) * (1.0*c/from_counts[m])
        adjacency_matrix[n] = edges_n
    
    # We assigned the unallocated probability into a self-loop
    for n, edges in adjacency_matrix.iteritems():
        total_n = sum(edges.values())
        edges[n] = 1-total_n
        adjacency_matrix[n] = edges

    return adjacency_matrix


import pprint 
def test_gv():

    message_counts = [
        ("a", "b", 50),
        ("a", "c", 2500),
        ("b", "c", 2500),
        ("b", "d", 250),
        ("c", "a", 250),
        ("c", "b", 250),
        ("c", "d", 250),
        ("d", "c", 500),
        ("d", "a", 20),
    ]
    
    adj = get_good_vibes_adjacency_matrix(message_counts)
    
    pprint.pprint(adj)

test_gv()

{'a': {'a': 0.5510992275698159,
       'b': 0.00326797385620915,
       'c': 0.44563279857397503},
 'b': {'b': 0.5413223140495869,
       'c': 0.4132231404958677,
       'd': 0.045454545454545456},
 'c': {'a': 0.30864197530864196,
       'b': 0.2777777777777778,
       'c': 0.24691358024691368,
       'd': 0.16666666666666666},
 'd': {'a': 0.002849002849002849,
       'c': 0.08741258741258742,
       'd': 0.9097384097384097}}


In [7]:
# node_probs is a dictionary { "node": probability, ...} with the probabilities on round k
# revised_probs is a dictionary { "node": probability, ... } with the probabilities on round k+1
# for the structure of the adjacency matrix, look above 
def propagate(node_probs, adjacency, smoothing_probs, d):
    revised_probs = dict()
    for n, p_n in node_probs.iteritems():
        # The edge_probs is a dictionary { "node": probability, ...} 
        # that contains the probability that a vibe is propagated
        # from n to m
        propagation_probs_from_n = adjacency.get(n)
        if propagation_probs_from_n == None:
            continue
        for m, p_n_m in propagation_probs_from_n.iteritems():
            p_m = revised_probs.get(m)
            if p_m == None:
                p_m = 0
            p_m += p_n * p_n_m
            revised_probs[m] = p_m
    
    return add_vectors(revised_probs, smoothing_probs, 1-d, d)

In [8]:
def computePagerank(adjacency, d, max_iterations, max_diff):
    nodes = set(adjacency.keys())
    smoothing = dict()
    for n in nodes:
        smoothing[n] = 1.0/len(nodes) 
    pagerank = smoothing
    
    for i in range(0,max_iterations):
        pagerank_new = propagate(pagerank, adjacency, smoothing, d)
        # print pagerank_new
        # check for convergence
        diff = sum(diff_vectors(pagerank_new, pagerank).values())
        if diff < max_diff:
            break
        else:
            pagerank = pagerank_new
        
    return pagerank

In [9]:
def computePersonalizedPagerank(adjacency, start, d, max_iterations, max_diff):
    
    ppr = {start:1.0}
    smoothing_probs = {start:1.0}
    for i in range(0,max_iterations):
        ppr_new = propagate(ppr, adjacency, smoothing_probs, d)
        # print ppr_new
        # check for convergence
        diff = sum(diff_vectors(ppr_new, ppr).values())
        if diff < max_diff:
            break
        else:
            ppr = ppr_new
    
    return ppr

In [10]:
def load_matrix(filename):
    f = open("data/msg_counts.csv", "r")
    lines = f.read().split("\n")
    
    msg_counts = []
    # skip the header line and the last line (because it is empty, and I am lazy to remove it)
    for line in lines[1: len(lines)-2]:
        fields = line.split("\t")
        # Tuple = sender, recipient, count
        t = (str(int(fields[0])), str(int(fields[1])), float(fields[2]) )
        msg_counts.append(t)
    return get_good_vibes_adjacency_matrix(msg_counts)


In [11]:
# This is a test adjacency matrix,
#adjacency_matrix = {"a": {"b": 0.5, "c":0.25, "a":0.25}, 
#              "b": {"c": 0.25, "d":0.25, "b": 0.5}, 
#              "c": {"a": 0.25, "b": 0.25, "d": 0.25, "c": 0.25},
#              "d": {"c":0.25, "a":0.1, "d": 0.65}
#             }

adjacency_matrix = {"a": {"b": 0.333, "f": 0.333, "g": 0.333}, 
              "b": {"c": 1.0}, 
              "c": {"d": 1.0},
              "d": {"e": 1.0},
              "e": {"e": 1.0},
              "f": {"c": 1.0},
              "g": {"c": 1.0}  
             }

d = 0.8
max_iterations = 30
max_diff = 0.00001
start_node = "a"

# Computes the personalized pagerank of start_node
ppr = computePersonalizedPagerank(adjacency_matrix, start_node, d, max_iterations, max_diff)
print "PERSONALIZED PAGERANK for", start_node, "\n", ppr
norm_ppr = remove_normalize(ppr, start_node)
print "RENORMALIZED PERSONALIZED PAGERANK for", start_node, "\n", norm_ppr

PERSONALIZED PAGERANK for a 
{'a': 0.8, 'c': 0.03196799999999999, 'b': 0.053279999999999994, 'e': 0.0015983999999999987, 'd': 0.006393599999999997, 'g': 0.053279999999999994, 'f': 0.053279999999999994}
RENORMALIZED PERSONALIZED PAGERANK for a 
{'c': 0.15983999999999998, 'b': 0.2664, 'e': 0.007991999999999996, 'd': 0.03196799999999999, 'g': 0.2664, 'f': 0.2664}


In [13]:
def testpagerank(): 
    
    filename = "data/msg_counts.csv"
    adjacency_matrix = load_matrix(filename)

    d = 0.7
    max_iterations = 30
    max_diff = 0.0000000001
    start_node = "33618"
    # start_node = "021950"

    # Computes the pagerank of each node
    # Should be run just once, as it does not change
    pr = computePagerank(adjacency_matrix, d, max_iterations, max_diff)
    # print "PAGERANK\n", pr  

    # Computes the personalized pagerank of start_node
    ppr = computePersonalizedPagerank(adjacency_matrix, start_node, d, max_iterations, max_diff)
    # print "PERSONALIZED PAGERANK for", start_node, "\n", ppr
    
    # Since we want to compute the connections of start_node, we renormalize the vector
    # of probabilities to exclude the start_node, and allocate the probability proportionally
    # to the other nodes. Now we have two sets of probabilities. One is from general pagerank
    # which captures the prominence of the node in the network, and one is from personalized
    # pagerank which illustrates the prominence of each node wrt to start_node
    norm_pr = remove_normalize(pr, start_node)
    norm_ppr = remove_normalize(ppr, start_node)
    # print "RENORMALIZED PAGERANK after removing", start_node, "\n", norm_pr
    # print "RENORMALIZED PERSONALIZED PAGERANK for", start_node, "\n", norm_ppr
    
    
    # We now compute the log-odds of each node. Positive values mean higher association
    # with the start node compared to the expectation (Value of 1 means "twice as likely compared to
    # expectation", value 2 means four times as likely, etc.)
    log_odds = logodds_vector(norm_ppr, norm_pr, 1)
    for k, v in log_odds.iteritems():
        # We print only nodes with "significantly" positive scores ("significantly" above 0.0)
        if v>0.01:
            print k, round(v,4)

    # print "LOG-ODDS score for", start_node, "\n",log_odds
    
testpagerank()

21950 0.9903
37821 0.0152
